In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/github/Dacon/항공편지연예측AI경진대회(2023.04.03-2023.05.08)')
os.getcwd()

'/content/drive/MyDrive/github/Dacon/항공편지연예측AI경진대회(2023.04.03-2023.05.08)'

## 1. 결측치처리
**해당 노트북**
+ 전처리방법2 + x결측치삭제 + vae 활용 + validation set 확인

In [2]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

### 1.1. 전처리방법2 데이터 가져오기

In [3]:
train = pd.read_parquet('./data/train_preprocess_2.parquet')
# test = pd.read_parquet('./test.parquet')
test = pd.read_parquet('./data/test_preprocess_2.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

print(train.shape)
print(train.Delay.value_counts())

(1000000, 19)
Not_Delayed    210001
Delayed         45000
Name: Delay, dtype: int64


### 1.2. 남은 결측치 처리 - 삭제

In [4]:
# print(train.isnull().sum())
# print(train.dropna().shape)
# print(train.dropna().isnull().sum())
train = train.dropna(subset=['Estimated_Departure_Time','Estimated_Arrival_Time','Carrier_Code(IATA)','Airline','Carrier_ID(DOT)'])
print(train.isnull().sum())


# 레이블(Delay)을 제외한 결측값이 존재하는 변수들을 unknown으로 대체합니다.
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    # mode = train[col].mode()[0]
    # train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna('Unknown')
print('Done.')

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time         0
Estimated_Arrival_Time           0
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)               0
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       520399
dtype: int64
Done.


In [74]:
# test.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,Unknown,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,19977.0,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,Georgia,746.0,Delta Air Lines Inc.,DL,19790.0,N3765
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT


In [75]:
# 새로운 column 생성
# train['Estimated_Duration'] = train['Estimated_Arrival_Time'] -  train['Estimated_Departure_Time']
# test['Estimated_Duration'] = test['Estimated_Arrival_Time'] -  test['Estimated_Departure_Time']
test['Tail_Number'].nunique()

6493

### 1.3. label & unlabel split  / label_train & label_validation split

#### 1배치 데이터 흐름
1. vae에는 X_train_labeled와 X_unlabeled를 각각 onehot으로 만들어서 합쳐서 넣어주기
2. classifier에는 X_train_labeled를 onehot으로 만든 것 넣어주기


#### 필요한 것
1. labeled와 unlabeled 나누기
2. labeled에서 train과 validation 분리하기
3. X_train_labeld & X_unlabeled 를 이용한 onehot encoding
4. 전체 데이터에 onehot 적용하면 데이터 크기 너무 커지므로, 배치로 처리하기

#### 1.3.1. 데이터 쪼개기

In [13]:
# 1. labeled & unlabeld split
train_labeled , train_unlabeled = train[train['Delay'].notnull()], train[train['Delay'].isnull()]

X_labeled, y_labeled = train_labeled.drop(['ID','Delay'], axis=1), train_labeled[['Delay']]
change_cate2num = {'Not_Delayed':0, "Delayed":1}
y_labeled['Delay'] = y_labeled['Delay'].apply(lambda x : change_cate2num[x])
X_unlabeled = train_unlabeled.drop(['ID','Delay'], axis=1)

print(X_labeled.shape, X_unlabeled.shape)


# 2. train_labeled & val_labeled split
from sklearn.model_selection import train_test_split
X_train_labeled, X_val_labeled, y_train_labeled, y_val_labeled = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)


# 3. X_unlabled 의 크기를 X_train_labeled와 맞춰주기
X_unlabeled = X_unlabeled.iloc[:len(X_train_labeled),:]
print(X_unlabeled.shape, X_train_labeled.shape )

(178176, 17) (520399, 17)
(142540, 17) (142540, 17)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3696\410560725.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_labeled['Delay'] = y_labeled['Delay'].apply(lambda x : change_cate2num[x])


#### 1.3.2. encoder 만들기

In [23]:

# 3. 데이터 정리 & onehotencoding
from sklearn.preprocessing import OneHotEncoder
cate_cols = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number']


# Airport 2개 삭제함
cate_cols = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 
       'Origin_Airport_ID', 'Origin_State', 
       'Destination_Airport_ID', 'Destination_State', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number']

cate_cols = ['Month', 'Day_of_Month',
       'Origin_Airport_ID',
       'Destination_Airport_ID', 
             'Airline']

numeric_cols = ['Estimated_Departure_Time','Estimated_Arrival_Time','Distance']


## 3.1. VAE 훈련에 쓸 데이터 : X_train_labeled, X_unlabeled
### 3.1.1. 데이터 정리
X_vae_train = pd.concat([X_train_labeled, X_unlabeled])
X_vae_train_cate = X_vae_train[cate_cols]

encoder = OneHotEncoder()
encoder.fit(X_vae_train_cate)


OneHotEncoder()

### 1.4. 데이터 만들기

In [56]:
import numpy as np
from torch.utils.data.dataset import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch

class Flight_labeled(Dataset): 
    def __init__(self, X_train_labeled, y_train_labeled, encoder):
        # 1. 데이터 받아오기
        self.X_train_labeled = X_train_labeled
        self.y_train_labeled =  y_train_labeled
        
        self.cate_cols = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport_ID', \
                          'Origin_State', 'Destination_Airport_ID', 'Destination_State', 'Airline',\
                          'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number']
        
        self.cate_cols = cate_cols

        self.numeric_cols = ['Estimated_Departure_Time','Estimated_Arrival_Time','Distance']

        self.max_values = X_train_labeled[self.numeric_cols].max()
        
        
    # 사용 가능한 데이터 개수 return
    def __len__(self):
        return len(self.X_train_labeled)
    
    def __getitem__(self, i):
        # onehot encoding 후, tensor로 반환하기
        # 1. category는 onehot으로 변환하고, numeric은 category onehot 뒤에 붙이기
        X_sample_category = self.X_train_labeled[self.cate_cols].iloc[i,:].to_frame().T
#         print('1: ', X_sample_category.dtype, X_sample_category.shape)
        
        X_sample_category = encoder.transform(X_sample_category)
#         print('2: ', X_sample_category.dtype, X_sample_category.shape)
        
        X_sample_category = X_sample_category.toarray()  # 추가된 코드: X_sample_category를 2차원 배열로 변환 : 희소행렬로 반환되는 onehot encoding 결과를, 일반적인 Numpy 배열로 변환해줌
#         print('3: ', X_sample_category.dtype, X_sample_category.shape)
        
        X_sample_numeric = np.array(self.X_train_labeled[self.numeric_cols].iloc[i,:]).reshape(1,-1)
        X_sample_numeric = X_sample_numeric / self.max_values.values.reshape(1,-1)

#         print('4: ', X_sample_numeric.dtype, X_sample_numeric.shape)
        
        X_sample = np.hstack([X_sample_category,X_sample_numeric])
#         print('5: ', X_sample.dtype, X_sample.shape)
        
        # 2. 텐서로 변환하기
        X_sample = torch.tensor(X_sample.squeeze(), dtype=torch.float32) # (1, input_dim) -> (input_dim,) 으로 차원 축소
        y_sample = torch.tensor(self.y_train_labeled.iloc[i,:].values, dtype=torch.float32)
        

        return X_sample, y_sample

   


class Flight_unlabeled(Dataset):
    def __init__(self, X_unlabeled, encoder):
        self.X_unlabeled =  X_unlabeled
        
        self.cate_cols = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport_ID', \
                          'Origin_State', 'Destination_Airport_ID', 'Destination_State', 'Airline',\
                          'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number']
        
        self.cate_cols = cate_cols

        self.numeric_cols = ['Estimated_Departure_Time','Estimated_Arrival_Time','Distance']

        self.max_values = X_unlabeled[self.numeric_cols].max()
        
        
    # 사용 가능한 데이터 개수 return
    def __len__(self):
        return len(self.X_unlabeled)
    
    def __getitem__(self, i):
        # onehot encoding 후, tensor로 반환하기
        # 1. category는 onehot으로 변환하고,
        X_sample_category = self.X_unlabeled[self.cate_cols].iloc[i,:].to_frame().T
        
        X_sample_category = encoder.transform(X_sample_category)
        
        X_sample_category = X_sample_category.toarray()  # 추가된 코드: X_sample_category를 2차원 배열로 변환 : 희소행렬로 반환되는 onehot encoding 결과를, 일반적인 Numpy 배열로 변환해줌
        
        # 2.  numeric은 0~1 사이로 만들어서, category onehot 뒤에 붙이기
        X_sample_numeric = np.array(self.X_unlabeled[self.numeric_cols].iloc[i,:]).reshape(1,-1)
        X_sample_numeric = X_sample_numeric / self.max_values.values.reshape(1,-1)
        
        X_sample = np.hstack([X_sample_category,X_sample_numeric])
        
        # 3. 텐서로 변환하기
        X_sample = torch.tensor(X_sample.squeeze(), dtype=torch.float32)
        return X_sample
    


In [51]:
# 데이터 생성
from torch.utils.data.dataloader import DataLoader
import tqdm

# 데이터 가져오기
label_dataset = Flight_labeled(X_train_labeled,y_train_labeled, encoder)
unlabel_dataset = Flight_unlabeled(X_unlabeled, encoder)
print(len(label_dataset),len(unlabel_dataset))

# 배치화 : unlabel의 batch 개수를 label의 배치개수에 맞춰주기
label_loader = DataLoader(label_dataset, batch_size=1024, num_workers=0)

# unlabel_batch_size = len(unlabel_dataset)//len(label_loader) +1
# print('unlabel batch size : ', unlabel_batch_size )
unlabel_loader = DataLoader(unlabel_dataset, batch_size=1024, num_workers=0)
print(len(label_loader),len(unlabel_loader))


142540 142540
140 140


In [64]:
# 데이터 최소, 최대값 확인
for batch_idx, (data, _) in enumerate(label_loader):
    data = data.float()
    print(data.shape)
    print(data.min(), data.max())


torch.Size([1024, 823])
tensor(0.) tensor(1.)


KeyboardInterrupt: 

In [45]:
# 데이터 잘 만들어졌는지 확인
iterator = tqdm.tqdm(label_loader)
for data, label in iterator:
    print(len(data), label.sum())
    print("Data batch shape:", data.shape)
    print("Label batch shape:", label.shape)

  1%|▌                                                                                 | 1/140 [00:05<12:27,  5.38s/it]

1024 tensor(189.)
Data batch shape: torch.Size([1024, 823])
Label batch shape: torch.Size([1024, 1])


  1%|█▏                                                                                | 2/140 [00:10<12:25,  5.41s/it]

1024 tensor(183.)
Data batch shape: torch.Size([1024, 823])
Label batch shape: torch.Size([1024, 1])


  1%|█▏                                                                                | 2/140 [00:14<17:04,  7.43s/it]


KeyboardInterrupt: 

### 모델생성

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Define the VAE model
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = torch.chunk(h, 2, dim=1)
        z = self.reparameterize(mu, log_var)
        x_hat = self.decoder(z)
        return x_hat, mu, log_var




In [31]:
# 분류기 생성
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)


In [53]:
# Hyperparameters
input_dim = label_dataset[0][0].shape[0]
hidden_dim = 256
latent_dim = 128
epochs = 1
learning_rate = 0.001

# Initialize the model, optimizer and loss function
device = 'cuda:0'
model = VAE(input_dim, hidden_dim, latent_dim)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
reconstruction_loss = nn.BCELoss(reduction='mean')


# classifier 초기화
classifier = Classifier(input_dim = latent_dim, hidden_dim = latent_dim//2 , output_dim= 2)
classifier.to(device)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Classifier(
  (network): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=2, bias=True)
    (5): Softmax(dim=1)
  )
)

In [27]:
# unlabel Training loop 확인

# for epoch in range(epochs):
#     iterator = tqdm.tqdm(unlabel_loader)
#     for data in iterator:
# #         print(data.shape) # shape = [batch_size, input_dim]
#         optimizer.zero_grad()
#         reconstructed_data, mu, log_var = model(data.to(device))
#         loss = reconstruction_loss(reconstructed_data, data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
#         loss.backward()
#         optimizer.step()
        
#         iterator.set_description(f"epoch:{epoch+1}, loss : {loss.item()}")



epoch:1, loss : 566470.9375:   1%|▊                                                    | 2/140 [00:11<12:53,  5.60s/it]


KeyboardInterrupt: 

In [55]:
# label Training loop 확인
import warnings
warnings.filterwarnings('ignore')

for epoch in range(epochs):
    iterator = tqdm.tqdm(label_loader)
    for data, label in iterator:
#         print(data.shape) # shape = [batch_size, input_dim]
        optimizer.zero_grad()
        reconstructed_data, mu, log_var = model(data.to(device))
        loss = reconstruction_loss(reconstructed_data, data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss.backward()
        optimizer.step()
        
        iterator.set_description(f"epoch:{epoch+1}, loss : {loss.item()}")


epoch:1, loss : 163.43850708007812:   1%|▎                                           | 1/140 [00:27<1:03:43, 27.51s/it]


KeyboardInterrupt: 

In [40]:
# unlabeled 과 train_labeled data 같이 이용해서 신경망 훈련
classification_loss = nn.CrossEntropyLoss()

for epoch in range(1):
    iterator = tqdm.tqdm(zip(label_loader, unlabel_loader), total=len(label_loader))
    
    for (labeled_data, labeled_label), unlabeled_data in iterator:
        optimizer.zero_grad()

        # Labeled data에 대한 VAE 훈련
#         print(labeled_data.shape)
        reconstructed_labeled_data, mu, log_var = model(labeled_data.to(device))
        labeled_loss = reconstruction_loss(reconstructed_labeled_data, labeled_data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Unlabeled data에 대한 VAE 훈련
#         print(unlabeled_data.shape)
        reconstructed_unlabeled_data, mu, log_var = model(unlabeled_data.to(device))
        unlabeled_loss = reconstruction_loss(reconstructed_unlabeled_data, unlabeled_data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Labeled data에 대한 Classifier 훈련
        latent_labeled_data = model.reparameterize(mu, log_var)
        classifier_output = classifier(latent_labeled_data)
        labeled_label_indices = torch.argmax(labeled_label, dim=1).to(device) # 다중 타겟을 원핫으로 변환
        class_loss = classification_loss(classifier_output, labeled_label_indices)
        
        # 총 Loss 계산 및 업데이트
        total_loss = labeled_loss + unlabeled_loss + class_loss
        total_loss.backward()
        optimizer.step()

        iterator.set_description(f"epoch:{epoch+1}, labeled_loss: {labeled_loss.item().round(4)}, unlabeled_loss: {unlabeled_loss.item().round(4)}, class_loss: {class_loss.item().round(4)}")


epoch:1, labeled_loss: 0.0540984570980072, unlabeled_loss: 0.05303141474723816, class_loss: 0.7160175442695618: 100%|█|


In [48]:
# unlabeled 과 train_labeled data 같이 이용해서 신경망 훈련
classification_loss = nn.CrossEntropyLoss()

for epoch in range(1):
    iterator = tqdm.tqdm(zip(label_loader, unlabel_loader), total=len(label_loader))
    
    for (labeled_data, labeled_label), unlabeled_data in iterator:
        optimizer.zero_grad()

        # Labeled data에 대한 VAE 훈련
#         print(labeled_data.shape)
        reconstructed_labeled_data, mu, log_var = model(labeled_data.to(device))
        labeled_loss = reconstruction_loss(reconstructed_labeled_data, labeled_data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Unlabeled data에 대한 VAE 훈련
#         print(unlabeled_data.shape)
        reconstructed_unlabeled_data, mu, log_var = model(unlabeled_data.to(device))
        unlabeled_loss = reconstruction_loss(reconstructed_unlabeled_data, unlabeled_data.to(device)) - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Labeled data에 대한 Classifier 훈련
        latent_labeled_data = model.reparameterize(mu, log_var)
        classifier_output = classifier(latent_labeled_data)
        labeled_label_indices = torch.argmax(labeled_label, dim=1).to(device) # 다중 타겟을 원핫으로 변환
        class_loss = classification_loss(classifier_output, labeled_label_indices)
        
        # 총 Loss 계산 및 업데이트
        total_loss = labeled_loss + unlabeled_loss + class_loss
        total_loss.backward()
        optimizer.step()

        iterator.set_description(f"epoch:{epoch+1}, total:{total_loss.item() : .4f}, labeled_loss:{labeled_loss.item() : .4f}, unlabeled_loss:{unlabeled_loss.item() : .4f}, class_loss: {class_loss.item(): .4f}")


epoch:1, total: 0.7844, labeled_loss: 0.0343, unlabeled_loss: 0.0338, class_loss:  0.7163: 100%|█| 140/140 [25:13<00:00


In [52]:
torch.save(model.state_dict(), './VAE.pth') # 모델 저장
torch.save(classifier.state_dict(), './classifier.pth') # 모델 저장

In [20]:
encoder.categories_

# 

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       dtype=int64),
 array([0], dtype=int64),
 array([0], dtype=int64),
 array([10135, 10136, 10140, 10141, 10146, 10154, 10155, 10157, 10158,
        10165, 10170, 10185, 10208, 10245, 10257, 10268, 10275, 10279,
        10299, 10333, 10361, 10372, 10397, 10408, 10409, 10423, 10431,
        10434, 10466, 10469, 10529, 10551, 10558, 10561, 10562, 10577,
        10581, 10599, 10620, 10627, 10631, 10643, 10666, 10676, 10685,
        10693, 10713, 10721, 10728, 10731, 10732, 10739, 10747, 10754,
        10779, 10781, 10785, 10792, 10800, 10821, 10849, 10868, 10874,
        10918, 10926, 10967, 10980, 10990, 10994, 11003, 11013, 11027,
        11042, 11049, 11057, 11066, 11067, 11076, 11092, 11097, 11109,
        11111, 11122, 11140, 11146, 11150, 11193, 11203, 11233, 11252,
     

## 아래는 아직 작성 안한 부분

In [34]:
y_labeled

,Delay
6,Not_Delayed
8,Not_Delayed
10,Delayed
12,Not_Delayed
13,Not_Delayed
...,...
999950,Not_Delayed
999955,Delayed
999963,Delayed
999985,Not_Delayed


## 2. 준지도학습진행

### 전처리방법7 저장

In [ ]:
save_idx = input('몇 번째 전처리 방법인지 정수-정수를 입력하세요 : ')
train_save_name = 'train_preprocess_' + save_idx
test_save_name = 'test_preprocess_' + save_idx
train.to_parquet(f'./data/{train_save_name}.parquet')
test.to_parquet(f'./data/{test_save_name}.parquet')

몇 번째 전처리 방법인지 정수-정수를 입력하세요 : 4
